In [3]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [4]:
import numpy as np
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    y = np.zeros_like(vec)
    y[0]=np.linalg.norm(vec)
    u = (vec - y)/np.linalg.norm(vec-y)
    H = np.eye(vec.shape[0]) - 2*np.outer(u, u)

    return H@vec, H

Test your function using tests below:

In [5]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [6]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [13]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    Q = np.eye(m) 
    
    for i in range(min(m-1, n)):
        vec, H_i = householder(a1[i:, i]) 
        E = np.eye(i)
        O = np.zeros((i, m-i))
        O1 = np.zeros((m-i, i))
        Q_shtr_i =np.asarray(np.bmat([[E, O], [O1, H_i]])) 
        a1 = Q_shtr_i @ a1 
        Q = Q @ Q_shtr_i.T 
        print()             
    return Q, a1
    

In [27]:
x = np.array([[-1,1, 2], [9, 5, 1], [1, 7, 1]])
print("x = ",x)
print("Q = ", qr_decomp(x)[0])
print("R = ", qr_decomp(x)[1])
print("QR = ", qr_decomp(x)[0] @ qr_decomp(x)[1])
print("x-QR", x- qr_decomp(x)[0] @ qr_decomp(x)[1])

x =  [[-1  1  2]
 [ 9  5  1]
 [ 1  7  1]]


Q =  [[-0.10976426  0.24432711  0.96346047]
 [ 0.98787834 -0.08022681  0.1328911 ]
 [ 0.10976426  0.96636842 -0.23255942]]


R =  [[ 9.11043358  5.59797726  0.87811408]
 [-0.          6.60777198  1.37479583]
 [-0.         -0.          1.82725261]]




QR =  [[-1.  1.  2.]
 [ 9.  5.  1.]
 [ 1.  7.  1.]]




x-QR [[-0. -0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -0.]]


Работает

In [8]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [9]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [25]:
from scipy.linalg import qr
qq, rr = qr(a)

print("Q = ", qr_decomp(a)[0])
print("R = ", qr_decomp(a)[1])
print("QQ = ",qq, "\n")
print("RR = ",rr)
print("RR-R", abs(rr)-abs(qr_decomp(a)[1]))
print("QQ-Q", abs(qq)-abs(qr_decomp(a)[0]))
assert_allclose(np.dot(qq, rr), a)






Q =  [[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]



R =  [[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.         -0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]
QQ =  [[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 

RR =  [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.        

Отличие матриц можно обьяснить тем, что QR разложение не является единсвенным

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [28]:
def constr_R(a): 
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    U = np.zeros_like(a1)
    
    for i in range(min(m-1, n)):
        R = a1[i:, i:]
        vec = R[:, 0]
        
        y = np.zeros_like(vec)
        y[0]=np.linalg.norm(vec)
        u = (vec - y)/np.linalg.norm(vec-y) 
        R = R - 2*np.outer(u, u.T @ R) 
        a1[i:, i:] = R 
        U[:len(vec),i]= u 
    return(a1, U)
                        
print("Библиотечная: ",qr(x)[1], "\n")

def constr_R_Q_U(a, U): 
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    for i in range(min(m-1, n)): 
        R = a1[i:, i:]
       
        u = U[:R.shape[0],i]
        R = R - 2*np.outer(u, (u.T) @ R) 
        a1[i:, i:] = R 
    return a1
print("Моя реализация: ",constr_R_Q_U(x, constr_R(x)[1]),"\n")
print("Разность:",abs(qr(x)[1])-abs(constr_R_Q_U(x, constr_R(x)[1])) )


Библиотечная:  [[ 9.11043358  5.59797726  0.87811408]
 [ 0.         -6.60777198 -1.37479583]
 [ 0.          0.         -1.82725261]] 

Моя реализация:  [[9.11043358 5.59797726 0.87811408]
 [0.         6.60777198 1.37479583]
 [0.         0.         1.82725261]] 

Разность: [[ 0.  0. -0.]
 [ 0.  0. -0.]
 [ 0.  0.  0.]]


Отличие только в знаке